In [1]:
!pip install MTCNN

  Obtaining dependency information for MTCNN from https://files.pythonhosted.org/packages/09/d1/2a4269e387edb97484157b872fa8a1953b53dcafbe4842a1967f549ac5ea/mtcnn-0.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for opencv-python>=4.1.0 from https://files.pythonhosted.org/packages/3f/a4/d2537f47fd7fcfba966bd806e3ec18e7ee1681056d4b0a9c8d983983e4d5/opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 3.3 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 5.9 MB/s eta 0:00:00m eta 0:00:010:00:01


In [17]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50
import numpy as np
import cv2
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
from mtcnn import MTCNN
import os


In [18]:
# Initialize the face detector (MTCNN) and feature extractor (ResNet50)
detector = MTCNN()

# Initialize the model
model = TripletResNet50(embedding_dim=128)

# Load the model's state dictionary
model.load_state_dict(torch.load('./triplet_resnet50_cpu.pth'))

# Set the model to evaluation mode
model.eval()


NameError: name 'TripletResNet50' is not defined

In [8]:
# Preprocessing pipeline for images
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [9]:
#!pip install kaggle

In [10]:
#!mkdir -p ~/.kaggle

In [11]:
#!kaggle datasets download -d jessicali9530/celeba-dataset

In [12]:
# Path to your small dataset of celebrity images
celeba_img_path = './celeb_dataset/img_align_celeba/img_align_celeba/'  # Replace with your small dataset directory

# Load image file paths from the small dataset
celeba_image_files = os.listdir(celeba_img_path)
celeba_image_paths = [os.path.join(celeba_img_path, img_file) for img_file in celeba_image_files]



In [13]:
#print(celeba_image_files)

In [14]:
print("Total number of files", len(celeba_image_files))

Total number of files 202599


In [15]:
# Function to extract face embeddings from an image
def extract_face_embedding(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect face in the image
    detection = detector.detect_faces(image_rgb)

    if len(detection) > 0:
        x, y, w, h = detection[0]['box']
        face = image_rgb[y:y+h, x:x+w]
        face = Image.fromarray(face)
        face_tensor = preprocess(face).unsqueeze(0)

        with torch.no_grad():
            embedding = model(face_tensor).numpy()

        return embedding
    else:
        print(f"No face detected in {image_path}")
        return None

# Function to compare the user image against a small set of celebrity images
def find_celebrity_look_alike(user_image_path, celeb_image_paths, top_n=3):
    user_embedding = extract_face_embedding(user_image_path)

    if user_embedding is None:
        return "No face detected in the user image."

    similarities = []

    # Compare with each celebrity image
    for celeb_path in celeb_image_paths:
        celeb_embedding = extract_face_embedding(celeb_path)
        if celeb_embedding is not None:
            similarity = cosine_similarity(user_embedding, celeb_embedding)[0][0]
            similarities.append((celeb_path, similarity))

    # Sort celebrities by similarity (highest first)
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    # Return the top N most similar celebrities
    return similarities[:top_n] if similarities else "No face detected in any celebrity images."


In [16]:
# Step 1: Compute celebrity embeddings and save to a file

celeb_embeddings = {}
for celeb_path in celeba_image_paths:
    celeb_name = os.path.basename(celeb_path)
    embedding = extract_face_embedding(celeb_path)
    if embedding is not None:
        celeb_embeddings[celeb_name] = embedding


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step


NameError: name 'model' is not defined

In [3]:
import pickle
# Save embeddings to a file (using pickle or numpy)
with open('celeb_embeddings.pkl', 'wb') as f:
    pickle.dump(celeb_embeddings, f)

print(f"Celebrity embeddings saved successfully!")

NameError: name 'celeb_embeddings' is not defined